In [42]:
# import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [148]:
# store url for page with list of us birds
listURL = "https://en.wikipedia.org/wiki/List_of_birds_of_the_United_States"
# get html
listResult=requests.get(listURL)
# create beautiful soup object
soup = bs(listResult.text, 'html.parser')

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of birds of the United States - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_birds_of_the_United_States","wgTitle":"List of birds of the United States","wgCurRevisionId":825853558,"wgRevisionId":825853558,"wgArticleId":14941202,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use American English from July 2017","All Wikipedia articles written in American English","Lists of birds by country","Birds of the United States","Lists of biota of the United States"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentMod

In [203]:
# find all a objects
a = soup.findAll('a')

# create subsetted list of those that do not contain the word title
a2 = []
for item in a:
    if re.search(r"title", str(item)) !=None:
        a2.append(item)

#create subsetted list of those that do contain the word class
a3 = []
for item in a2:
    if re.search(r"class", str(item)) == None:
        a3.append(item)

# create subsetted list of those that do not contain the word wiki
a4 = []
for item in a3:
    if re.search(r"wiki", str(item)) !=None:
        a4.append(item)

# create empty list
titles = []
# if the entry has a title, append to titles
for item in a4:
    try:
        titles.append(item['title'])
    except KeyError:
        pass

# write to file
fh = open("usBirds.txt", "w")
for item in titles:
    fh.write("%s\n" % item.encode("utf-8"))
fh.close()

[<li class="toclevel-1 tocsection-1"><a href="#Sources_and_geographic_coverage"><span class="tocnumber">1</span> <span class="toctext">Sources and geographic coverage</span></a></li>,
 <li class="toclevel-1 tocsection-2"><a href="#Taxonomy_and_status"><span class="tocnumber">2</span> <span class="toctext">Taxonomy and status</span></a></li>,
 <li class="toclevel-1 tocsection-3"><a href="#By_the_numbers"><span class="tocnumber">3</span> <span class="toctext">By the numbers</span></a></li>,
 <li class="toclevel-1 tocsection-4"><a href="#Ducks,_geese,_and_waterfowl"><span class="tocnumber">4</span> <span class="toctext">Ducks, geese, and waterfowl</span></a></li>,
 <li class="toclevel-1 tocsection-5"><a href="#Chachalacas,_curassows,_and_guans"><span class="tocnumber">5</span> <span class="toctext">Chachalacas, curassows, and guans</span></a></li>,
 <li class="toclevel-1 tocsection-6"><a href="#Guineafowl"><span class="tocnumber">6</span> <span class="toctext">Guineafowl</span></a></li>,


In [107]:
# fxn to scrape image, audio & information urls from the Cornell Bird Lab site
def img_audio(bird):
    # chrome driver
    executable_path = {'executable_path': 'chromedriver'}
    browser = Browser('chrome', **executable_path, headless=True)

    # replace spaces with underscores
    bird = bird.replace(" ","_")

    # open search page
    url = f'https://www.allaboutbirds.org/guide/{bird}'
    browser.visit(url)

    # store html
    html = browser.html
    
    # create beautiful soup object
    soup = bs(html, 'html.parser')

    # check title to make sure the page is for the desired bird
    # and not a search result because the bird couldn't be found
    title = soup.find('title').text
    titleR = re.search(r'Search Results for', title)
    if titleR == None:
        
        # check for audio clip
        jp = soup.find('audio', {"id": "jp_audio_0"})
        if jp == None:
            audio = 'no audio'
        else:
            audio = jp['src']
       
        # check if image exists
        hero = soup.find('section', {"class": "hero-wrap"})
        if hero == None:
            img = 'no image'
        else:
            img_style = hero['style']
            img_re = re.search(r'"https\:\/\/(.+)"', img_style)
            img = img_re.group(1)
            
        # store link
        link = url
    else:
        audio = 'no audio'
        img = 'no img'
        link = 'no link'

    # return list of the image & audio files & link
    return [img, audio, link]

In [222]:
# read in list of species to scrape images, audio & info links
speciesList = pd.read_csv("scrapingList_US2.csv", encoding='Latin-1')
speciesList.head()

,COMMON_NAME
0,Barrows goldeneye
1,Gambels quail
2,Rosss goose
3,Stellers eider
4,Barn swallow


In [223]:
# store name of file to log downloads
csvName = "scraped.csv"

# for each bird
for index, row, in speciesList['COMMON_NAME'].iteritems():
    # print the bird name to track progress
    print(row)
    # run the img_audio fxn to scrape urls
    results = img_audio(row)
    # print urls
    print(results)
    # create a dictionary with the results
    newRow = {"COMMON_NAME": row, "image": results[0], "audio": results[1], "link": results[2]}
    # convert it to a df
    newRow = pd.DataFrame(newRow, index=[0])
    # append it to the csv
    with open(csvName, 'a') as f:
        newRow.to_csv(f, header=False, index = False)

Barrows goldeneye
['s3.amazonaws.com/com-aab-media/photo/75259221-1280px.jpg', 'https://s3.amazonaws.com/com-aab-media/sound/521893.mp3', 'https://www.allaboutbirds.org/guide/Barrows_goldeneye']
Gambels quail
['s3.amazonaws.com/com-aab-media/photo/75360871-1280px.jpg', 'https://s3.amazonaws.com/com-aab-media/sound/521913.mp3', 'https://www.allaboutbirds.org/guide/Gambels_quail']
Rosss goose
['s3.amazonaws.com/com-aab-media/photo/75368001-1280px.jpg', 'https://s3.amazonaws.com/com-aab-media/sound/521828.mp3', 'https://www.allaboutbirds.org/guide/Rosss_goose']
Stellers eider
['s3.amazonaws.com/com-aab-media/photo/75239601-1280px.jpg', 'https://s3.amazonaws.com/com-aab-media/sound/528094.mp3', 'https://www.allaboutbirds.org/guide/Stellers_eider']
Barn swallow
['s3.amazonaws.com/com-aab-media/photo/75259341-1280px.jpg', 'https://s3.amazonaws.com/com-aab-media/sound/522686.mp3', 'https://www.allaboutbirds.org/guide/Barn_swallow']
Black rail
['s3.amazonaws.com/com-aab-media/photo/75226851-12

In [266]:
# fxn to scrape image, audio & information urls from the Internet Bird Collection
def img_ibc(bird):
    executable_path = {'executable_path': 'chromedriver'}
    browser = Browser('chrome', **executable_path, headless=True)

    # open search page
    url = 'https://www.hbw.com/ibc/species'
    browser.visit(url)

    # fill in search bar & execute search
    browser.find_by_xpath('//*[@id="edit-ibc-search-string"]').fill(bird)
    browser.find_by_xpath('//*[@id="edit-submit-button"]').first.click()

    # store html
    html = browser.html

    # create beautiful soup object
    soup = bs(html, 'html.parser')

    # store first h1 with title class
    title = soup.find('h1', {'class': 'title'})
    
    # check title to make sure the page is for the desired bird
    # and not a search result because the bird couldn't be found
    if title.text == 'Search in the Internet Bird Collection':
        link = 'no link'
        img = 'no image'
    
    else: 
        # check if image exists
        img_a = soup.find('a', {"class": "colorbox"})
        if img_a == None:
                img = 'no image'
        else:
            img_i = img_a.find('img')
            img = img_i['src']

        # check if taxonomy link exists
        link_div = soup.find('div', {"class": "ibc-species-info"})
        if link_div == None:
            link = 'no link'
        else:
            link_a = link_div.findAll('a')[2]
            link = 'https://www.hbw.com' + link_a['href']

    return([img, link])
    

In [248]:
# read in file with list of birds still needing image urls
speciesList = pd.read_csv("scrapingList_US3_noCornell.csv", encoding='Latin-1')
speciesList.head()

,COMMON_NAME
0,Adelaides warbler
1,African silverbill
2,African stonechat
3,Aleutian tern
4,Amazon kingfisher


In [267]:
# store name of file to log downloads
csvName = "scraped2.csv"

# for each bird
for index, row, in speciesList['COMMON_NAME'].iteritems():
    print(row)
    results = img_ibc(row)
    print(results)
    newRow = {"COMMON_NAME": row, "image": results[0], "link": results[1]}
    newRow = pd.DataFrame(newRow, index=[0])
    scraped = scraped.append(newRow, ignore_index = True)

    with open(csvName, 'a') as f:
        newRow.to_csv(f, header=False, index = False)

Adelaides warbler
['no image', 'no link']
African silverbill
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/b21a6645c.jpg?itok=nfYOiOIo', 'https://www.hbw.com/species/african-silverbill-euodice-cantans#Taxonomy']
African stonechat
['no image', 'no link']
Aleutian tern
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/1683_20160531_2_1000.jpg?itok=K3Ol6yvN', 'https://www.hbw.com/species/aleutian-tern-onychoprion-aleuticus#Taxonomy']
Amazon kingfisher
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/123a1059_cloroceryle_amazona_samica.jpg?itok=d_93i5yX', 'https://www.hbw.com/species/amazon-kingfisher-chloroceryle-amazona#Taxonomy']
American flamingo
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/IMG_0811_0.jpg?itok=YK0pFtSk', 'https://www.hbw.com/species/american-flamingo-phoenicopterus-ruber#Taxonomy']
Antillean nighthawk
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Antillean_Nighthawk2

['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/bf6e1334_brown_shrike.jpg?itok=vLC75WFN', 'https://www.hbw.com/species/brown-shrike-lanius-cristatus#Taxonomy']
Brown-backed solitaire
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Myadestes_occidentalis_6JH.jpg?itok=cdGrrKs3', 'https://www.hbw.com/species/brown-backed-solitaire-myadestes-occidentalis#Taxonomy']
Brown-chested martin
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Gray-breasted_Martin.jpg?itok=V3XDcNwp', 'https://www.hbw.com/species/brown-chested-martin-progne-tapera#Taxonomy']
Brown-crested flycatcher
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/brown-crested_flycatcher_dusan_brinkhuizen_ibc.jpg?itok=fdbHMYEg', 'https://www.hbw.com/species/brown-crested-flycatcher-myiarchus-tyrannulus#Taxonomy']
Bryans shearwater
['no image', 'no link']
Budgerigar
['no image', 'no link']
Bullers shearwater
['no image', 'no link']
Bulwers petrel
['no image

['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/IMG_0475.JPG?itok=6nfSYB6b', 'https://www.hbw.com/species/eared-quetzal-euptilotis-neoxenus#Taxonomy']
Eastern yellow wagtail
['no image', 'no link']
Elf owl
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/elf_owl_04.jpg?itok=FcpLaGep', 'https://www.hbw.com/species/elf-owl-micrathene-whitneyi#Taxonomy']
Erckels francolin
['no image', 'no link']
Eskimo curlew
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/charscola6811.jpg?itok=qEvY9mlA', 'https://www.hbw.com/species/eskimo-curlew-numenius-borealis#Taxonomy']
Eurasian bullfinch
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/img_2949.jpg?itok=i6_CbVEJ', 'https://www.hbw.com/species/eurasian-bullfinch-pyrrhula-pyrrhula#Taxonomy']
Eurasian coot
['no image', 'no link']
Eurasian curlew
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/1560_20160121_1_1000.jpg?itok=KobFuQ0Y', 'https://www.hbw.com/

['no image', 'no link']
Kalij pheasant
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/860058571512c8dc9c173c.jpg?itok=qTUErRvx', 'https://www.hbw.com/species/kalij-pheasant-lophura-leucomelanos#Taxonomy']
Kamchatka leaf warbler
['no image', 'no link']
Kauai elepaio
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Kauai_Elepaio_c_Jim_Denny_IBC.jpg?itok=8aXUbYAQ', 'https://www.hbw.com/species/kauai-elepaio-chasiempis-sclateri#Taxonomy']
Kelp gull
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/bagre.jpg?itok=ZeT0hYC7', 'https://www.hbw.com/species/kelp-gull-larus-dominicanus#Taxonomy']
Key West quail-dove
['no image', 'no link']
Kioea
['no image', 'no link']
Kittlitzs murrelet
['no image', 'no link']
Kona grosbeak
['no image', 'https://www.hbw.com/species/kona-grosbeak-chloridops-kona#Descriptive_notes']
La Sagras flycatcher
['no image', 'no link']
Lanai hookbill
['no image', 'https://www.hbw.com/species/lanai-hookbill-dysmorodr

['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/717A5982FOROS.jpg?itok=n9H8Hcxg', 'https://www.hbw.com/species/orange-billed-nightingale-thrush-catharus-aurantiirostris#Taxonomy']
Orange-cheeked waxbill
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/bf6e2739_orange-cheeked_waxbill.jpg?itok=Ybz7_z_4', 'https://www.hbw.com/species/orange-cheeked-waxbill-estrilda-melpoda#Taxonomy']
Oriental cuckoo
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/1o3a7946hopfkuckucka.jpg?itok=OnUHmw5A', 'https://www.hbw.com/species/oriental-cuckoo-cuculus-saturatus#Taxonomy']
Oriental pratincole
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/1635_oriental_pratincole_glareola_maldivarum_haha-jima_ogasawara_islands_japan_20170527_1_1000.jpg?itok=pmb6tdEk', 'https://www.hbw.com/species/oriental-pratincole-glareola-maldivarum#Taxonomy']
Pacific swift
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/3105_pacific_

['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/9863_saffron_finch_sicalis_flaveola_zaruma_el_oro_ecuador_20131219_1_1600.jpg?itok=X9xAaYgy', 'https://www.hbw.com/species/saffron-finch-sicalis-flaveola#Taxonomy']
Salvins albatross
['no image', 'no link']
Scaly-breasted munia
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Scaly-breasted_Munia_Lonchura_punctulata2.jpg?itok=PU6JzUh9', 'https://www.hbw.com/species/scaly-breasted-munia-lonchura-punctulata#Taxonomy']
Scaly-naped pigeon
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/patagioenas_squamosa_m_4.jpg?itok=PagpjzRe', 'https://www.hbw.com/species/scaly-naped-pigeon-patagioenas-squamosa#Taxonomy']
Scarlet ibis
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/123a1726_scarlet_ibis.jpg?itok=NcWScMus', 'https://www.hbw.com/species/scarlet-ibis-eudocimus-ruber#Taxonomy']
Scrippss murrelet
['no image', 'no link']
Sedge warbler
['https://www.hbw.com/sites/defau

['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/_M4A3135.jpg?itok=ocKxBU2K', 'https://www.hbw.com/species/wedge-tailed-shearwater-ardenna-pacifica#Taxonomy']
Western spindalis
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/western_spindalis_bahamas.jpg?itok=J4XvF888', 'https://www.hbw.com/species/western-spindalis-spindalis-zena#Taxonomy']
Whiskered auklet
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Whiskered_Auklet_Plymalka_Aethia_pygmaea.jpg?itok=ypb3kc8i', 'https://www.hbw.com/species/whiskered-auklet-aethia-pygmaea#Taxonomy']
Whiskered tern
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/img_1017_0.jpg?itok=H0IcPfd5', 'https://www.hbw.com/species/whiskered-tern-chlidonias-hybrida#Taxonomy']
White wagtail
['no image', 'no link']
White-capped albatross
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/f2030003g.jpg?itok=dD3NADY0', 'https://www.hbw.com/species/white-capped-albatross-

['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/1o3a0986helmperlhuhn_ssp._mitratusa.jpg?itok=37_T52WM', 'https://www.hbw.com/species/helmeted-guineafowl-numida-meleagris#Taxonomy']
Labrador Duck
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/mlc.2011.0.309_a.jpg?itok=8E--7ThF', 'https://www.hbw.com/species/labrador-duck-camptorhynchus-labradorius#Descriptive_notes']
Laysan Duck
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/laysan-duck-1a.jpg?itok=PyERRKzb', 'https://www.hbw.com/species/laysan-duck-anas-laysanensis#Taxonomy']
Lesser White-fronted Goose
['no image', 'no link']
Masked Duck
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Pato_fierro_3.jpg?itok=mENxm9Cu', 'https://www.hbw.com/species/masked-duck-nomonyx-dominicus#Taxonomy']
Pink-footed Goose
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/89__ar7q9357.jpg?itok=TCghO35M', 'https://www.hbw.com/species/pink-footed-goose-anser

In [268]:
# read in file with list of birds still needing image urls
speciesList = pd.read_csv("scrapingList_US4_reformat.csv", encoding='Latin-1')
speciesList.head()

,COMMON_NAME
0,Adelaide's warbler
1,Audubon's shearwater
2,Bachman's sparrow
3,Bachman's warbler
4,Belcher's gull


In [269]:
# store name of file to log downloads
csvName = "scraped3.csv"

# for each bird
for index, row, in speciesList['COMMON_NAME'].iteritems():
    print(row)
    results = img_ibc(row)
    print(results)
    newRow = {"COMMON_NAME": row, "image": results[0], "link": results[1]}
    newRow = pd.DataFrame(newRow, index=[0])
    scraped = scraped.append(newRow, ignore_index = True)

    with open(csvName, 'a') as f:
        newRow.to_csv(f, header=False, index = False)

Adelaide's warbler
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/Adelaides_warbler.jpg?itok=Q7Wr34N7', 'https://www.hbw.com/species/adelaides-warbler-setophaga-adelaidae#Taxonomy']
Audubon's shearwater
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/audubonsshearwater_undflt_nsmyrna_fl_9_25_16_a_am8i8675.jpg?itok=1FJCpvLe', 'https://www.hbw.com/species/audubons-shearwater-puffinus-lherminieri#Taxonomy']
Bachman's sparrow
['https://www.hbw.com/sites/default/files/styles/h_340/public/ibc/p/bachmanssparrow_peer_tree_oken_ga_3_10_16_A_A61T1797.jpg?itok=mXRRXsaj', 'https://www.hbw.com/species/bachmans-sparrow-peucaea-aestivalis#Taxonomy']
Bachman's warbler
['no image', 'https://www.hbw.com/species/bachmans-warbler-vermivora-bachmanii#Taxonomy']
Belcher's gull
['no image', 'no link']
Bishop's oo
['no image', 'https://www.hbw.com/species/bishops-oo-moho-bishopi#Descriptive_notes']
Blyth's reed warbler
['no image', 'no link']
Bryan's shearwater
['no im

In [299]:
# fxn to scrape image, audio & information urls from the Internet Bird Collection
def img_wiki(bird):
    executable_path = {'executable_path': 'chromedriver'}
    browser = Browser('chrome', **executable_path, headless=True)

    # open search page
    url = 'https://www.wikipedia.org'
    browser.visit(url)

    # fill in search bar & execute search
    browser.find_by_xpath('//*[@id="searchInput"]').fill(bird)
    browser.find_by_xpath('//*[@id="search-form"]/fieldset/button').first.click()

    # store html
    html = browser.html

    # create beautiful soup object
    soup = bs(html, 'html.parser')

    # store heading
    title = soup.find('h1', {'id': 'firstHeading'})
    # compare to bird name to make sure it returned the desired result
    if title.text.lower() == bird.lower():
        
        img_table = soup.find('table', {'class': 'infobox'})
        if img_table == None:
            img = 'no image'
        else:
            img_a = img_table.find('a')
            img_img = img_a.find('img')
            if img_img == None:
                img = 'no image'
            else:
                img = img_img['src']
            
        link = browser.url
    else:
        img = 'no image'
        link = 'no link'

    return([img, link])

In [294]:
# read in file with list of birds still needing image urls
speciesList = pd.read_csv("scrapingList_US5_wiki.csv", encoding='Latin-1')
speciesList.head()

,COMMON_NAME,Done
0,Barolo shearwater,African stonechat
1,Black francolin,Asian brown flycatcher
2,Black kite,Bananaquit
3,Blue mockingbird,NaN
4,Brambling,NaN


In [300]:
# store name of file to log downloads
csvName = "scraped4.csv"

# for each bird
for index, row, in speciesList['COMMON_NAME'].iteritems():
    print(row)
    results = img_wiki(row)
    print(results)
    newRow = {"COMMON_NAME": row, "image": results[0], "link": results[1]}
    newRow = pd.DataFrame(newRow, index=[0])
    scraped = scraped.append(newRow, ignore_index = True)

    with open(csvName, 'a') as f:
        newRow.to_csv(f, header=False, index = False)

Barolo shearwater
['no image', 'https://en.wikipedia.org/wiki/Barolo_shearwater']
Black francolin
['//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Black_Francolin.jpg/220px-Black_Francolin.jpg', 'https://en.wikipedia.org/wiki/Black_francolin']
Black kite
['//upload.wikimedia.org/wikipedia/commons/thumb/5/5a/Schwarzmilan.jpg/220px-Schwarzmilan.jpg', 'https://en.wikipedia.org/wiki/Black_kite']
Blue mockingbird
['//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Blue_Mockingbird.jpg/220px-Blue_Mockingbird.jpg', 'https://en.wikipedia.org/wiki/Blue_mockingbird']
Brambling
['//upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Fringilla_montifringilla_-Poland_-male-8.jpg/220px-Fringilla_montifringilla_-Poland_-male-8.jpg', 'https://en.wikipedia.org/wiki/Brambling']
Budgerigar
['no image', 'https://en.wikipedia.org/wiki/Budgerigar']
Cape Verde shearwater
['no image', 'https://en.wikipedia.org/wiki/Cape_Verde_shearwater']
Cinnamon hummingbird
['//upload.wikimedia.org/wikipedia/commons/th

['//upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Luscinia_cyane_-_Khao_Yai.jpg/220px-Luscinia_cyane_-_Khao_Yai.jpg', 'https://en.wikipedia.org/wiki/Siberian_blue_robin']
Smew
['//upload.wikimedia.org/wikipedia/commons/thumb/4/47/Zwergsaeger_maenchen_weibchen.jpg/220px-Zwergsaeger_maenchen_weibchen.jpg', 'https://en.wikipedia.org/wiki/Smew']
South polar skua
['//upload.wikimedia.org/wikipedia/commons/thumb/a/a3/South_polar_skua.jpg/220px-South_polar_skua.jpg', 'https://en.wikipedia.org/wiki/South_polar_skua']
Spotted dove
['//upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Spotted_dove_%28Spilopelia_chinensis_suratensis%29.jpg/220px-Spotted_dove_%28Spilopelia_chinensis_suratensis%29.jpg', 'https://en.wikipedia.org/wiki/Spotted_dove']
Sungrebe
['//upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Sungrebe.jpg/220px-Sungrebe.jpg', 'https://en.wikipedia.org/wiki/Sungrebe']
Taiga flycatcher
['//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Red-breasted_Flycatcher.jpg/220px-Red-br

In [302]:
test = {'comName': ['x', 'y','z'], 'comName2': ['x','y','z']}
test.keys()

dict_keys(['comName', 'comName2'])